In [1]:
import algorithm as al
import numpy as np
import cv2 as cv
import Queue
from patchdb import *
from psi import *
from copyutils import *
from debug import *
import inspect

[INFO              ] [Logger      ] Record log in C:\Users\xiaoy\.kivy\logs\kivy_16-04-06_5.txt
[INFO              ] [Kivy        ] v1.9.1
[INFO              ] [Python      ] v2.7.11 |Continuum Analytics, Inc.| (default, Feb 16 2016, 09:58:36) [MSC v.1500 64 bit (AMD64)]
[INFO              ] [Image       ] Providers: img_tex, img_dds, img_gif, img_sdl2 (img_pil, img_ffpyplayer ignored)


In [267]:
def imshow(img):
    cv.imshow('image', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
_p = al.Inpainting()

In [4]:
_p.readImage('../../test_images/input-color.jpg', 'source')

_p.readImage('../../test_images/input-alpha.bmp', 'alpha')

(True, 'Success')

In [5]:
_p._images['filled'] = np.uint8(_p._images['alpha'] > 0)*255
_p._images['inpainted'] = _p._images['source'].copy()

In [6]:
for i in range(0,3):
    _p._images['inpainted'][:,:,i] *= (_p._images['filled']>0)

#
# Step 1a,b: Identify the fill front deltaOmega and compute
#            initial patch priorities

In [7]:
_p.computeBoundaries()
_p.confidenceInitialize()

In [8]:
_p._w = 5

In [9]:
_p._patchDB = PatchDB(_p._images['inpainted'], _p._w, filled=_p._images['filled'])

In [10]:
_p.iterationsInit()
done = False

In [11]:
boundary = _p._boundaryIterator.next()
_p.fillFrontInitialize(boundary, imviewer=None)

In [117]:
boundaryPixels = boundary
# initialize the pixels on the fill front
_p._images['fillFront'] = \
    np.zeros_like(_p._images['filled'], dtype=np.uint8)
_p._images['fillFront'] = \
    cv.drawContours(_p._images['fillFront'],boundaryPixels,-1,255)

# initialize the priority queue with all points on the fill front
_p._deltaOmega = Queue.PriorityQueue()

In [118]:
col, row = boundaryPixels[boundaryPixels[:,0,1] > 306][0][0]
p = PSI((row, col), _p._w, 
          image=_p._images['inpainted'], 
          filled=_p._images['filled'],
          confidence=_p._images['confidence'],
          fillFront=_p._images['fillFront'])

In [126]:
cwindow, _ = copyutils.getWindow(_p._images['confidence'], p._coords, p._w)
filled, valid = copyutils.getWindow(_p._images['filled'], p._coords, p._w)
psiPArea = valid.sum()
C = cwindow[filled>0].sum() / psiPArea

In [120]:
# Sobel filter
v1 = np.array([1,2,1])
v2 = np.array([-1,0,1])
# Sobel filter for the x-derivative
sobelxcol = v1[:,np.newaxis]
sobelxrow = v2[np.newaxis,:]
# Sobel filter for the y-derivative
sobelycol = v2[:,np.newaxis]
sobelyrow = v1[np.newaxis,:]

In [216]:
pixels, _ = copyutils.getWindow(_p._images['inpainted'], p._coords, 1)

In [217]:
pixels

array([[[24, 19, 21],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[24, 19, 21],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[24, 19, 21],
        [ 0,  0,  0],
        [ 0,  0,  0]]], dtype=uint8)

In [219]:
Dx = sobelxrow * -1 * pixels * sobelxcol
Dy = sobelycol * -1 * pixels * sobelyrow

In [225]:
Dx

array([[[ 24,   0, -21],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 24,   0, -21],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 24,   0, -21],
        [  0,   0,   0],
        [  0,   0,   0]]])

In [258]:
p2 = _p._images['inpainted'].sum(axis=2)/3

In [259]:
_p._images['inpainted'][1][1]

array([22,  8, 26], dtype=uint8)

In [269]:
imshow(p2)

TypeError: mat data type = 6 is not supported

In [271]:
gray_image = cv.cvtColor(_p._images['inpainted'], cv.COLOR_BGR2GRAY)

In [273]:
gray_image[0][0]

20

In [276]:
imshow(_p._images['filled'])

In [279]:
(_p._images['filled'] ==0).sum()

8887